In [1]:
%config IPCompleter.greedy=True


In [2]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
from sklearn.utils import class_weight 
import matplotlib
import matplotlib.pyplot as plt

# own code
import sys
sys.path.append("..") # relative path to module toolkit
from toolkit import getLabelsFromDir

Using TensorFlow backend.


In [3]:
# setup
input_size = (224, 224, 3)
num_classes = 44
n = 34
mini_batch_size = 32 #article takes 256
epochs = 60 * 10 ** 4

model_type = 'ResNet%dv1' % (n)
save_dir = os.path.join(os.getcwd(), 'saved_models_weighted')
model_name = 'resnetOrig_%s_model.{epoch:03d}.h5' % model_type

train_dir = "../../images/images_genus/train/"
val_dir = "../../images/images_genus/val/"
train_images = 12525
val_images = 3454

save_plot_name = "resnetOrig_weighted_trainplot.png"

In [4]:
steps_per_epoch = int(train_images/mini_batch_size) + 1
validation_steps = int(val_images/mini_batch_size) + 1

In [5]:
def resnet_layer(inputs,
                 num_filters=64,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


# Model architecture as in "Deep Residual Learning for Image Recognition"

In [6]:
def resnet_v1(input_shape, depth, num_classes):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    stage 2:  8x8,  512

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes

    # Returns
        model (Model): Keras model instance
    """

    # Start model definition.
    num_filters = 64
    
    if depth == 34:
        residual_blockSize = [3, 4, 6, 3] # used for depth 34 models 
    else:
        raise Exception("only depth 34 is available")
    not_first_layer = False
    
    #init first layer
    inputs = Input(shape=input_shape)
    
    x = resnet_layer(inputs,num_filters=num_filters, kernel_size=7, strides=2, activation=None)
    x = MaxPooling2D(pool_size=2, strides=2)(x)
    x = Activation('relu')(x)
    
    
    # Instantiate the stack of residual units
    for blockSize in residual_blockSize:
        for res_block in range(blockSize):
            strides = 1
            if not_first_layer and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if not_first_layer and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                
                # Equivalent to option B in paper, where only projection shortcuts are used for 
                # dimensionality(number of filters) increase.
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            elif not(not_first_layer):
                not_first_layer = True
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    #print(x.shape)
    
    y = GlobalAveragePooling2D()(x)

    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [7]:
model = resnet_v1(input_shape=input_size, depth=n, num_classes=num_classes)

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001,momentum=0.9, decay=0.0001),
              #optimizer=SGD(lr=0.0001),
              metrics=['accuracy'])

model.summary()
print(model_type)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [9]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_reducer = ReduceLROnPlateau(factor=0.1,
                               cooldown=0,
                               verbose=1,
                               patience=0,
                               min_lr=0.5e-6)

early_stop = EarlyStopping(patience=2)

callbacks = [checkpoint, lr_reducer, early_stop]

In [10]:
assert(getLabelsFromDir(train_dir) == getLabelsFromDir(val_dir))
labels = getLabelsFromDir(train_dir)

train_datagen = ImageDataGenerator(rescale=1./255.)
val_datagen = ImageDataGenerator(rescale=1./255.)

#no augmentation other than rescaling pixels

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=mini_batch_size,
                                                    color_mode='rgb',
                                                    target_size=input_size[:2] )
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=mini_batch_size,
                                                    color_mode='rgb',
                                                    target_size=input_size[:2] )


Found 12263 images belonging to 44 classes.
Found 3381 images belonging to 44 classes.


In [11]:
W = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)

In [12]:
# Fit the model on the batches generated by generator.
history = model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    #verbose=1,
                    validation_data=val_generator,
                    validation_steps=validation_steps,
                    callbacks=callbacks,
                    class_weight=W)

Epoch 1/600000
 14/392 [>.............................] - ETA: 7:26 - loss: 6.6716 - acc: 0.1763

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['acc'], label='training accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.savefig(save_plot_name, bbox_inches='tight')
plt.show()